<a href="https://colab.research.google.com/github/zbooster/Landmark-Image-Classification/blob/main/landmark_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비


## Google 드라이브에서 Resize된 데이터 가져오기
압축을 풀어서 나의 colab 폴더에 둔다

In [3]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!gdown 1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va

Downloading...
From: https://drive.google.com/uc?id=1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va
To: /content/resizeds.zip
100% 3.09G/3.09G [00:18<00:00, 169MB/s]


In [5]:
!gdown 'https://drive.google.com/file/d/1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va/view?usp=sharing'

/usr/local/lib/python3.7/dist-packages/gdown/parse_url.py:39: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va
  url="https://drive.google.com/uc?id={}".format(file_id)
Downloading...
From: https://drive.google.com/file/d/1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va/view?usp=sharing
To: /content/view?usp=sharing
70.2kB [00:00, 446kB/s]


In [2]:
!wget 'https://drive.google.com/file/d/1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va/view?usp=sharing'

--2022-08-01 10:47:16--  https://drive.google.com/file/d/1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.68.101, 74.125.68.138, 74.125.68.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.68.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing.1’

view?usp=sharing.1      [ <=>                ]  65.75K   401KB/s    in 0.2s    

2022-08-01 10:47:17 (401 KB/s) - ‘view?usp=sharing.1’ saved [67328]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -R '/content/drive/MyDrive/DeepLearning_Datas/resizeds.zip' . # Google 드라이브의 데이터 위치를 찾아 수정해야함.
!unzip -qq '/content/resizeds.zip' -d './datasets'

In [ ]:
import os

original_dataset_dir = '/content/datasets/resizeds/Training'
classes_list = os.listdir(original_dataset_dir)

classes_list[:2], len(classes_list)

(['가재마을세종호반베르디움2단지아파트', '새뜸마을10단지더샵힐스테이트아파트'], 84)

# 데이터 정리

## 폴더 생성


In [ ]:
base_dir = './splitted'
os.mkdir(base_dir)

In [ ]:
import shutil

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)

for cls in classes_list:
  os.mkdir(os.path.join(train_dir, cls))
  os.mkdir(os.path.join(validation_dir, cls))

## 데이터 복사

In [ ]:
import math

original_dataset_dir = '/content/datasets/resizeds'

for cls in classes_list:
  train_path = os.path.join(original_dataset_dir, 'Training', cls)
  train_fnames = os.listdir(train_path)

  print("Train size(", cls, "): ", len(train_fnames))
  for fname in train_fnames:
    src = os.path.join(train_path, fname)
    dst = os.path.join(os.path.join(train_dir, cls), fname)
    shutil.copyfile(src, dst)

  val_path = os.path.join(original_dataset_dir, 'Validation', cls)
  val_fnames = os.listdir(val_path)
  print("Validation size(", cls, "): ", len(val_fnames))
  for fname in val_fnames:
    src = os.path.join(val_path, fname)
    dst = os.path.join(os.path.join(validation_dir, cls), fname)
    shutil.copyfile(src, dst)

Train size( 가재마을세종호반베르디움2단지아파트 ):  107
Validation size( 가재마을세종호반베르디움2단지아파트 ):  13
Train size( 새뜸마을10단지더샵힐스테이트아파트 ):  109
Validation size( 새뜸마을10단지더샵힐스테이트아파트 ):  13
Train size( 힐스테이트세종리버파크7단지 ):  105
Validation size( 힐스테이트세종리버파크7단지 ):  13
Train size( 연기향토박물관 ):  183
Validation size( 연기향토박물관 ):  22
Train size( 조치원자이아파트 ):  106
Validation size( 조치원자이아파트 ):  13
Train size( 성요한성당 ):  264
Validation size( 성요한성당 ):  32
Train size( 연기봉산동의향나무 ):  189
Validation size( 연기봉산동의향나무 ):  23
Train size( 전의향교 ):  159
Validation size( 전의향교 ):  19
Train size( 금강수변공원 ):  107
Validation size( 금강수변공원 ):  13
Train size( 해들마을6단지e편한세상세종리버파크 ):  109
Validation size( 해들마을6단지e편한세상세종리버파크 ):  13
Train size( 금남모인당 ):  184
Validation size( 금남모인당 ):  22
Train size( 새뜸1단지메이저시티푸르지오 ):  104
Validation size( 새뜸1단지메이저시티푸르지오 ):  12
Train size( 도램마을15단지힐스테이트아파트 ):  107
Validation size( 도램마을15단지힐스테이트아파트 ):  13
Train size( 금남남산영당 ):  185
Validation size( 금남남산영당 ):  23
Train size( 범지기마을7단지호반베르디움에코시티아파트 ):  109
Validation size( 범

# 학습준비

In [ ]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256
EPOCH = 30

In [ ]:
import torchvision.transforms as transforms

data_transforms = {
    'train': transforms.Compose([transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(), 
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    'val':  transforms.Compose([transforms.Resize([64, 64]),
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

data_dir = './splitted'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x),
                                 transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x],
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=2) for x in ['train', 'val']}     
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}      
class_name = image_datasets['train'].classes

In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 84)
resnet = resnet.to(DEVICE)

critetion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
ct = 0
for child in resnet.children():
  ct += 1
  if ct < 6:
    for param in child.parameters():
      param.requires_grad = False

In [ ]:
from pickle import bytes_types
import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  

  for epoch in range(1, num_epochs + 1):
    since = time.time()
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'train':
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss/dataset_sizes[phase]
      epoch_acc = running_corrects.double()/dataset_sizes[phase]

      print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  model.load_state_dict(best_model_wts)

  return model

In [ ]:
model_resnet50 = train_resnet(resnet, critetion, optimizer_ft,
                              exp_lr_scheduler, num_epochs=EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

train Loss: 1.2205, Acc: 0.6901
val Loss: 1.3398, Acc: 0.6975
Completed in 2m 14s
train Loss: 0.2849, Acc: 0.9097
val Loss: 0.5704, Acc: 0.8511
Completed in 2m 10s
train Loss: 0.1631, Acc: 0.9483
val Loss: 0.6662, Acc: 0.8225
Completed in 2m 8s
train Loss: 0.1386, Acc: 0.9575
val Loss: 0.4513, Acc: 0.8949
Completed in 2m 9s
train Loss: 0.1147, Acc: 0.9647
val Loss: 0.6756, Acc: 0.8670
Completed in 2m 9s
train Loss: 0.0853, Acc: 0.9731
val Loss: 0.5295, Acc: 0.8690
Completed in 2m 9s
train Loss: 0.0726, Acc: 0.9777
val Loss: 0.4704, Acc: 0.8823
Completed in 2m 8s
train Loss: 0.0374, Acc: 0.9885
val Loss: 0.2190, Acc: 0.9342
Completed in 2m 7s
train Loss: 0.0167, Acc: 0.9953
val Loss: 0.2286, Acc: 0.9348
Completed in 2m 7s
train Loss: 0.0116, Acc: 0.9965
val Loss: 0.1853, Acc: 0.9528
Completed in 2m 9s
train Loss: 0.0102, Acc: 0.9966
val Loss: 0.2152, Acc: 0.9461
Completed in 2m 9s
train Loss: 0.0083, Acc: 0.9980
val Loss: 0.1882, Acc: 0.9501
Completed in 2m 7s
train Loss: 0.0077, Acc: 0

In [ ]:
!cp '/content/resnet50.pt' '/content/drive/MyDrive/DeepLearning_Datas'